# Chapter 22: REST APIs

**Web Development Fundamentals**

REST (Representational State Transfer) is an architectural style for building web
APIs that leverages HTTP semantics. This notebook covers REST conventions, JSON
request/response handling, content negotiation, middleware patterns, error handling,
CORS, API versioning, and culminates in a complete in-memory CRUD API -- all built
with standard library tools on top of the WSGI foundation from the previous notebooks.

## REST Conventions: Resources, Methods, Status Codes

REST maps HTTP methods to CRUD operations on resources:

| Operation | Method | URL Pattern | Success Code | Body |
|-----------|--------|-------------|-------------|------|
| List all | GET | `/resources` | 200 OK | Collection |
| Create | POST | `/resources` | 201 Created | New resource |
| Read one | GET | `/resources/{id}` | 200 OK | Single resource |
| Replace | PUT | `/resources/{id}` | 200 OK | Updated resource |
| Update | PATCH | `/resources/{id}` | 200 OK | Updated resource |
| Delete | DELETE | `/resources/{id}` | 204 No Content | Empty |

Key principles:
- **Resources** are nouns (e.g., `/users`, `/posts`), not verbs
- **HTTP methods** express the action
- **Status codes** communicate the outcome
- **JSON** is the standard interchange format

In [ ]:
from dataclasses import dataclass
from http import HTTPStatus


@dataclass(frozen=True)
class RESTEndpoint:
    """Documents a REST API endpoint."""
    operation: str
    method: str
    url_pattern: str
    success_status: HTTPStatus
    has_request_body: bool
    has_response_body: bool


# Standard CRUD endpoints for a "books" resource
book_endpoints: list[RESTEndpoint] = [
    RESTEndpoint("List all", "GET", "/api/books",
                 HTTPStatus.OK, False, True),
    RESTEndpoint("Create", "POST", "/api/books",
                 HTTPStatus.CREATED, True, True),
    RESTEndpoint("Read one", "GET", "/api/books/{id}",
                 HTTPStatus.OK, False, True),
    RESTEndpoint("Full update", "PUT", "/api/books/{id}",
                 HTTPStatus.OK, True, True),
    RESTEndpoint("Partial update", "PATCH", "/api/books/{id}",
                 HTTPStatus.OK, True, True),
    RESTEndpoint("Delete", "DELETE", "/api/books/{id}",
                 HTTPStatus.NO_CONTENT, False, False),
]

print("=== REST Endpoints for 'books' Resource ===")
print(f"{'Operation':<17s} {'Method':<8s} {'URL':<22s} {'Status':<20s} {'Req Body':<10s} {'Resp Body'}")
print("-" * 90)
for ep in book_endpoints:
    status_str = f"{ep.success_status.value} {ep.success_status.phrase}"
    print(
        f"{ep.operation:<17s} {ep.method:<8s} {ep.url_pattern:<22s} "
        f"{status_str:<20s} {str(ep.has_request_body):<10s} {ep.has_response_body}"
    )

## JSON Request/Response Handling

REST APIs communicate using JSON. On the server side, this means:
- **Requests**: Parse JSON from the request body (`wsgi.input`)
- **Responses**: Serialize Python objects to JSON, set `Content-Type: application/json`

Below are helper functions that handle JSON serialization, content-type
validation, and error handling for malformed input.

In [ ]:
import json
import io
from http import HTTPStatus
from typing import Any, Callable, Iterable


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


class JSONParseError(Exception):
    """Raised when request body cannot be parsed as JSON."""
    pass


def read_json_body(environ: Environ) -> Any:
    """Read and parse JSON from the request body.

    Raises JSONParseError if the body is not valid JSON.
    Returns None if the body is empty.
    """
    try:
        content_length = int(str(environ.get("CONTENT_LENGTH", "0") or "0"))
    except ValueError:
        content_length = 0

    if content_length == 0:
        return None

    wsgi_input = environ.get("wsgi.input")
    if not wsgi_input or not hasattr(wsgi_input, "read"):
        return None

    raw: bytes = wsgi_input.read(content_length)  # type: ignore[union-attr]
    try:
        return json.loads(raw)
    except (json.JSONDecodeError, UnicodeDecodeError) as e:
        raise JSONParseError(f"Invalid JSON: {e}") from e


def json_response(
    data: Any,
    status: HTTPStatus,
    start_response: StartResponse,
    extra_headers: list[tuple[str, str]] | None = None,
) -> Iterable[bytes]:
    """Build a JSON response with proper headers."""
    body = json.dumps(data, indent=2, default=str).encode("utf-8")
    headers: list[tuple[str, str]] = [
        ("Content-Type", "application/json; charset=utf-8"),
        ("Content-Length", str(len(body))),
    ]
    if extra_headers:
        headers.extend(extra_headers)
    start_response(f"{status.value} {status.phrase}", headers)
    return [body]


def no_content_response(start_response: StartResponse) -> Iterable[bytes]:
    """Build a 204 No Content response (for DELETE)."""
    start_response(
        f"{HTTPStatus.NO_CONTENT.value} {HTTPStatus.NO_CONTENT.phrase}",
        [],
    )
    return []


# Test JSON parsing
print("=== Valid JSON ===")
valid_body = json.dumps({"title": "Python Cookbook", "author": "Beazley"}).encode()
env: Environ = {
    "CONTENT_LENGTH": str(len(valid_body)),
    "wsgi.input": io.BytesIO(valid_body),
}
parsed = read_json_body(env)
print(f"  Parsed: {parsed}")
print(f"  Type:   {type(parsed).__name__}")

print("\n=== Invalid JSON ===")
invalid_body = b"{not valid json}"
env_bad: Environ = {
    "CONTENT_LENGTH": str(len(invalid_body)),
    "wsgi.input": io.BytesIO(invalid_body),
}
try:
    read_json_body(env_bad)
except JSONParseError as e:
    print(f"  Error: {e}")

print("\n=== Empty Body ===")
env_empty: Environ = {"CONTENT_LENGTH": "0", "wsgi.input": io.BytesIO(b"")}
result = read_json_body(env_empty)
print(f"  Result: {result}")

## Content Negotiation: Accept and Content-Type Headers

Content negotiation allows clients and servers to agree on the data format:

- **`Accept`** (request header): What formats the client can handle
- **`Content-Type`** (request/response header): What format the body is in

A well-designed API checks the `Accept` header and returns `406 Not Acceptable`
if it cannot produce the requested format. It also validates the `Content-Type`
of incoming requests.

In [ ]:
from http import HTTPStatus


def parse_accept_header(accept: str) -> list[tuple[str, float]]:
    """Parse an Accept header into a list of (media_type, quality) tuples.

    Example: 'text/html, application/json;q=0.9, */*;q=0.1'
    Returns: [('text/html', 1.0), ('application/json', 0.9), ('*/*', 0.1)]
    """
    result: list[tuple[str, float]] = []
    for item in accept.split(","):
        parts = item.strip().split(";")
        media_type = parts[0].strip()
        quality = 1.0
        for param in parts[1:]:
            param = param.strip()
            if param.startswith("q="):
                try:
                    quality = float(param[2:])
                except ValueError:
                    quality = 0.0
        result.append((media_type, quality))
    # Sort by quality descending
    result.sort(key=lambda x: x[1], reverse=True)
    return result


def accepts_json(environ: dict[str, object]) -> bool:
    """Check if the client accepts JSON responses."""
    accept = str(environ.get("HTTP_ACCEPT", "*/*"))
    preferences = parse_accept_header(accept)
    for media_type, quality in preferences:
        if quality <= 0:
            continue
        if media_type in ("application/json", "*/*", "application/*"):
            return True
    return False


def has_json_content_type(environ: dict[str, object]) -> bool:
    """Check if the request body is JSON."""
    content_type = str(environ.get("CONTENT_TYPE", "")).lower()
    return "application/json" in content_type


# Parse various Accept headers
print("=== Parsing Accept Headers ===")
test_headers = [
    "application/json",
    "text/html, application/json;q=0.9",
    "text/html, application/xhtml+xml, */*;q=0.8",
    "application/xml;q=0.9, application/json;q=1.0",
    "text/plain;q=0.5, text/html;q=0.8",
]

for header in test_headers:
    parsed = parse_accept_header(header)
    accepts = accepts_json({"HTTP_ACCEPT": header})
    print(f"  Accept: {header}")
    print(f"    Parsed:  {parsed}")
    print(f"    JSON OK: {accepts}")
    print()

# Content-Type validation
print("=== Content-Type Validation ===")
test_content_types = [
    "application/json",
    "application/json; charset=utf-8",
    "text/plain",
    "application/x-www-form-urlencoded",
    "",
]

for ct in test_content_types:
    is_json = has_json_content_type({"CONTENT_TYPE": ct})
    print(f"  Content-Type: {ct or '(empty)':45s} -> JSON: {is_json}")

## Middleware Pattern: Wrapping WSGI Apps

Middleware is a WSGI app that wraps another WSGI app, adding cross-cutting
behavior (logging, authentication, error handling, CORS) without modifying
the inner application. Middleware follows the decorator/wrapper pattern:
it receives a request, optionally modifies it, calls the inner app, and
optionally modifies the response.

In [ ]:
import time
import json
from typing import Callable, Iterable
from http import HTTPStatus


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


class LoggingMiddleware:
    """Middleware that logs request method, path, status, and duration."""

    def __init__(self, app: WSGIApp) -> None:
        self._app = app
        self.log: list[str] = []  # Capture logs for demonstration

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        method = str(environ.get("REQUEST_METHOD", "?"))
        path = str(environ.get("PATH_INFO", "/"))
        start = time.perf_counter()

        captured_status = ""

        def logging_start_response(
            status: str, headers: list[tuple[str, str]]
        ) -> Callable[..., object]:
            nonlocal captured_status
            captured_status = status
            return start_response(status, headers)

        result = self._app(environ, logging_start_response)
        duration_ms = (time.perf_counter() - start) * 1000

        log_entry = f"{method} {path} -> {captured_status} ({duration_ms:.1f}ms)"
        self.log.append(log_entry)
        return result


class AuthMiddleware:
    """Middleware that checks for an Authorization header."""

    def __init__(self, app: WSGIApp, valid_tokens: set[str]) -> None:
        self._app = app
        self._valid_tokens = valid_tokens

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        auth_header = str(environ.get("HTTP_AUTHORIZATION", ""))

        if auth_header.startswith("Bearer "):
            token = auth_header[7:]
            if token in self._valid_tokens:
                # Authenticated -- pass through
                return self._app(environ, start_response)

        # Unauthorized
        body = json.dumps({"error": "Unauthorized", "detail": "Invalid or missing token"}).encode()
        start_response(
            f"{HTTPStatus.UNAUTHORIZED.value} {HTTPStatus.UNAUTHORIZED.phrase}",
            [
                ("Content-Type", "application/json"),
                ("Content-Length", str(len(body))),
                ("WWW-Authenticate", 'Bearer realm="api"'),
            ],
        )
        return [body]


# Simple inner app
def inner_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    body = json.dumps({"message": "Secret data"}).encode()
    start_response("200 OK", [
        ("Content-Type", "application/json"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


# Stack middleware: logging -> auth -> app
valid_tokens = {"secret-token-123", "admin-token-456"}
app = LoggingMiddleware(AuthMiddleware(inner_app, valid_tokens))


def call_with_auth(token: str | None, path: str = "/api/data") -> None:
    status_result = ""
    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal status_result
        status_result = s
        return lambda *a: None

    environ: Environ = {
        "REQUEST_METHOD": "GET",
        "PATH_INFO": path,
    }
    if token:
        environ["HTTP_AUTHORIZATION"] = f"Bearer {token}"

    body = b"".join(app(environ, sr))
    token_display = token[:15] + "..." if token else "(none)"
    print(f"  Token: {token_display:20s} -> {status_result:20s} | {body.decode()}")


print("=== Middleware Chain Tests ===")
call_with_auth(None)                     # No token
call_with_auth("wrong-token")            # Invalid token
call_with_auth("secret-token-123")       # Valid token
call_with_auth("admin-token-456")        # Another valid token

print("\n=== Request Log ===")
for entry in app.log:
    print(f"  {entry}")

## Error Handling: JSON Errors and Exception Mapping

A robust REST API returns structured JSON error responses with appropriate
status codes. An error-handling middleware can catch exceptions and convert
them to proper HTTP responses, preventing stack traces from leaking to clients.

In [ ]:
import json
import traceback
from http import HTTPStatus
from typing import Callable, Iterable


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


class APIError(Exception):
    """Base exception for API errors that map to HTTP status codes."""

    def __init__(
        self,
        status: HTTPStatus,
        message: str,
        detail: str | None = None,
    ) -> None:
        super().__init__(message)
        self.status = status
        self.message = message
        self.detail = detail


class NotFoundError(APIError):
    def __init__(self, resource: str, resource_id: str) -> None:
        super().__init__(
            HTTPStatus.NOT_FOUND,
            f"{resource} not found",
            f"{resource} with id '{resource_id}' does not exist",
        )


class ValidationError(APIError):
    def __init__(self, message: str, detail: str | None = None) -> None:
        super().__init__(HTTPStatus.UNPROCESSABLE_ENTITY, message, detail)


class ConflictError(APIError):
    def __init__(self, message: str) -> None:
        super().__init__(HTTPStatus.CONFLICT, message)


# Exception-to-status mapping for standard Python exceptions
EXCEPTION_STATUS_MAP: dict[type, HTTPStatus] = {
    ValueError: HTTPStatus.BAD_REQUEST,
    KeyError: HTTPStatus.NOT_FOUND,
    PermissionError: HTTPStatus.FORBIDDEN,
    NotImplementedError: HTTPStatus.NOT_IMPLEMENTED,
}


class ErrorHandlingMiddleware:
    """Middleware that catches exceptions and returns JSON error responses."""

    def __init__(self, app: WSGIApp, debug: bool = False) -> None:
        self._app = app
        self._debug = debug

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        try:
            return self._app(environ, start_response)
        except APIError as e:
            return self._error_response(e.status, e.message, e.detail, start_response)
        except Exception as e:
            # Map known exceptions to status codes
            status = EXCEPTION_STATUS_MAP.get(
                type(e), HTTPStatus.INTERNAL_SERVER_ERROR
            )
            detail = traceback.format_exc() if self._debug else None
            return self._error_response(status, str(e), detail, start_response)

    @staticmethod
    def _error_response(
        status: HTTPStatus,
        message: str,
        detail: str | None,
        start_response: StartResponse,
    ) -> Iterable[bytes]:
        error_data: dict[str, object] = {
            "error": status.phrase,
            "status": status.value,
            "message": message,
        }
        if detail:
            error_data["detail"] = detail

        body = json.dumps(error_data, indent=2).encode("utf-8")
        start_response(
            f"{status.value} {status.phrase}",
            [
                ("Content-Type", "application/json; charset=utf-8"),
                ("Content-Length", str(len(body))),
            ],
        )
        return [body]


# Test with various exceptions
def failing_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    path = str(environ.get("PATH_INFO", "/"))
    if path == "/not-found":
        raise NotFoundError("Book", "42")
    elif path == "/validation":
        raise ValidationError("Title is required", "Field 'title' cannot be empty")
    elif path == "/value-error":
        raise ValueError("Invalid page number")
    elif path == "/crash":
        raise RuntimeError("Unexpected server error")
    body = b'{"ok": true}'
    start_response("200 OK", [("Content-Type", "application/json"),
                               ("Content-Length", str(len(body)))])
    return [body]


wrapped = ErrorHandlingMiddleware(failing_app, debug=True)

for path in ["/ok", "/not-found", "/validation", "/value-error", "/crash"]:
    captured = ""
    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal captured
        captured = s
        return lambda *a: None

    body = b"".join(wrapped({"PATH_INFO": path, "REQUEST_METHOD": "GET"}, sr))
    parsed = json.loads(body)
    msg = parsed.get("message", parsed.get("ok", ""))
    print(f"GET {path:17s} -> {captured:30s} | {msg}")

## CORS: Cross-Origin Resource Sharing

Browsers enforce the **Same-Origin Policy**, blocking requests from one origin
(e.g., `http://frontend.com`) to a different origin (e.g., `http://api.com`).
CORS headers tell browsers which cross-origin requests are allowed.

Key CORS headers:
- `Access-Control-Allow-Origin`: Which origins can access the API
- `Access-Control-Allow-Methods`: Which HTTP methods are allowed
- `Access-Control-Allow-Headers`: Which request headers are allowed
- `Access-Control-Max-Age`: How long preflight results can be cached

Browsers send a **preflight** `OPTIONS` request before certain cross-origin
requests to check permissions.

In [ ]:
import json
from typing import Callable, Iterable
from http import HTTPStatus


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


class CORSMiddleware:
    """Middleware that adds CORS headers to responses."""

    def __init__(
        self,
        app: WSGIApp,
        allowed_origins: list[str] | None = None,
        allowed_methods: list[str] | None = None,
        allowed_headers: list[str] | None = None,
        max_age: int = 3600,
    ) -> None:
        self._app = app
        self._allowed_origins = set(allowed_origins or ["*"])
        self._allowed_methods = allowed_methods or [
            "GET", "POST", "PUT", "PATCH", "DELETE", "OPTIONS"
        ]
        self._allowed_headers = allowed_headers or [
            "Content-Type", "Authorization", "Accept"
        ]
        self._max_age = max_age

    def _get_cors_headers(self, origin: str) -> list[tuple[str, str]]:
        """Build CORS headers based on the request origin."""
        if "*" in self._allowed_origins:
            allow_origin = "*"
        elif origin in self._allowed_origins:
            allow_origin = origin
        else:
            return []  # Origin not allowed

        return [
            ("Access-Control-Allow-Origin", allow_origin),
            ("Access-Control-Allow-Methods", ", ".join(self._allowed_methods)),
            ("Access-Control-Allow-Headers", ", ".join(self._allowed_headers)),
            ("Access-Control-Max-Age", str(self._max_age)),
        ]

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        method = str(environ.get("REQUEST_METHOD", "GET"))
        origin = str(environ.get("HTTP_ORIGIN", ""))
        cors_headers = self._get_cors_headers(origin)

        # Handle preflight OPTIONS request
        if method == "OPTIONS" and cors_headers:
            start_response(
                f"{HTTPStatus.NO_CONTENT.value} {HTTPStatus.NO_CONTENT.phrase}",
                cors_headers,
            )
            return []

        # For normal requests, add CORS headers to the response
        def cors_start_response(
            status: str, headers: list[tuple[str, str]]
        ) -> Callable[..., object]:
            return start_response(status, headers + cors_headers)

        return self._app(environ, cors_start_response)


# Inner API app
def api_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    body = json.dumps({"data": "API response"}).encode()
    start_response("200 OK", [
        ("Content-Type", "application/json"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


# Wrap with CORS middleware
cors_app = CORSMiddleware(
    api_app,
    allowed_origins=["http://frontend.example.com", "http://localhost:3000"],
)


def test_cors(method: str, origin: str) -> None:
    captured_headers: list[tuple[str, str]] = []
    captured_status = ""

    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal captured_status, captured_headers
        captured_status = s
        captured_headers = h
        return lambda *a: None

    environ: Environ = {
        "REQUEST_METHOD": method,
        "PATH_INFO": "/api/data",
        "HTTP_ORIGIN": origin,
    }
    body = b"".join(cors_app(environ, sr))

    cors_related = {k: v for k, v in captured_headers if k.startswith("Access-Control")}
    print(f"\n  {method} from {origin or '(no origin)'}")
    print(f"    Status: {captured_status}")
    print(f"    CORS headers: {cors_related}")


print("=== CORS Middleware Tests ===")
test_cors("OPTIONS", "http://frontend.example.com")  # Preflight, allowed
test_cors("GET", "http://frontend.example.com")       # Normal, allowed
test_cors("GET", "http://evil.com")                   # Not allowed
test_cors("OPTIONS", "http://localhost:3000")          # Preflight, allowed

## API Versioning Strategies

As APIs evolve, breaking changes need to be managed. Common versioning strategies:

1. **URL path versioning**: `/api/v1/users` vs `/api/v2/users`
2. **Header versioning**: `Accept: application/vnd.myapi.v2+json`
3. **Query parameter versioning**: `/api/users?version=2`

URL path versioning is the most common and most explicit approach.

In [ ]:
import json
import re
from typing import Callable, Iterable


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


class VersionedAPI:
    """Router that dispatches to versioned API implementations."""

    def __init__(self) -> None:
        self._versions: dict[str, WSGIApp] = {}
        self._version_pattern = re.compile(r"^/api/(v\d+)(/.*)?$")

    def register_version(self, version: str, app: WSGIApp) -> None:
        """Register a WSGI app for a specific API version."""
        self._versions[version] = app

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        path = str(environ.get("PATH_INFO", "/"))
        match = self._version_pattern.match(path)

        if not match:
            body = json.dumps({
                "error": "Not Found",
                "message": "Use /api/v1/ or /api/v2/ prefix",
                "available_versions": sorted(self._versions.keys()),
            }).encode()
            start_response("404 Not Found", [
                ("Content-Type", "application/json"),
                ("Content-Length", str(len(body))),
            ])
            return [body]

        version = match.group(1)
        remaining_path = match.group(2) or "/"

        version_app = self._versions.get(version)
        if version_app is None:
            body = json.dumps({
                "error": f"API version '{version}' not supported",
                "available_versions": sorted(self._versions.keys()),
            }).encode()
            start_response("404 Not Found", [
                ("Content-Type", "application/json"),
                ("Content-Length", str(len(body))),
            ])
            return [body]

        # Rewrite PATH_INFO to strip the version prefix
        environ["PATH_INFO"] = remaining_path
        environ["API_VERSION"] = version
        return version_app(environ, start_response)


# V1 API: returns user as {"name": "..."}
def v1_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    path = str(environ.get("PATH_INFO", "/"))
    if path == "/users":
        data = {"users": [{"name": "Alice"}, {"name": "Bob"}]}
    else:
        data = {"version": "v1", "path": path}
    body = json.dumps(data).encode()
    start_response("200 OK", [("Content-Type", "application/json"),
                               ("Content-Length", str(len(body)))])
    return [body]


# V2 API: returns user as {"full_name": "...", "id": N} (breaking change)
def v2_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    path = str(environ.get("PATH_INFO", "/"))
    if path == "/users":
        data = {
            "users": [
                {"id": 1, "full_name": "Alice Smith", "email": "alice@example.com"},
                {"id": 2, "full_name": "Bob Jones", "email": "bob@example.com"},
            ],
            "total": 2,
        }
    else:
        data = {"version": "v2", "path": path}
    body = json.dumps(data).encode()
    start_response("200 OK", [("Content-Type", "application/json"),
                               ("Content-Length", str(len(body)))])
    return [body]


# Set up versioned API
api = VersionedAPI()
api.register_version("v1", v1_app)
api.register_version("v2", v2_app)

# Test versioned routing
def test_version(path: str) -> None:
    captured = ""
    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal captured
        captured = s
        return lambda *a: None

    body = b"".join(api({"REQUEST_METHOD": "GET", "PATH_INFO": path}, sr))
    data = json.loads(body)
    print(f"GET {path:25s} -> {captured:15s} | {json.dumps(data)}")


print("=== API Versioning ===")
test_version("/api/v1/users")
test_version("/api/v2/users")
test_version("/api/v3/users")  # Unsupported version
test_version("/other/path")    # Non-API path

## Practical: Building a Complete Mini REST API

Now we combine everything into a fully functional in-memory CRUD API for
managing "books". It includes:
- Proper routing with path parameters
- JSON request/response handling
- HTTP method dispatch
- Appropriate status codes
- Input validation
- Error handling

In [ ]:
import json
import re
import io
from http import HTTPStatus
from typing import Any, Callable, Iterable
from dataclasses import dataclass, field, asdict


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


# --- Domain Model ---

@dataclass
class Book:
    id: int
    title: str
    author: str
    year: int
    isbn: str = ""


class BookStore:
    """In-memory book storage with CRUD operations."""

    def __init__(self) -> None:
        self._books: dict[int, Book] = {}
        self._next_id: int = 1

    def list_all(self) -> list[Book]:
        return list(self._books.values())

    def get(self, book_id: int) -> Book | None:
        return self._books.get(book_id)

    def create(self, title: str, author: str, year: int, isbn: str = "") -> Book:
        book = Book(id=self._next_id, title=title, author=author, year=year, isbn=isbn)
        self._books[book.id] = book
        self._next_id += 1
        return book

    def update(self, book_id: int, **kwargs: Any) -> Book | None:
        book = self._books.get(book_id)
        if book is None:
            return None
        for key, value in kwargs.items():
            if hasattr(book, key) and key != "id":
                setattr(book, key, value)
        return book

    def delete(self, book_id: int) -> bool:
        return self._books.pop(book_id, None) is not None


# --- Request Helpers ---

def read_json(environ: Environ) -> Any:
    """Read JSON body from request."""
    try:
        length = int(str(environ.get("CONTENT_LENGTH", "0") or "0"))
    except ValueError:
        length = 0
    if length == 0:
        return None
    wsgi_input = environ.get("wsgi.input")
    if not wsgi_input or not hasattr(wsgi_input, "read"):
        return None
    raw = wsgi_input.read(length)  # type: ignore[union-attr]
    return json.loads(raw)


def respond_json(
    data: Any, status: HTTPStatus, start_response: StartResponse
) -> Iterable[bytes]:
    body = json.dumps(data, indent=2, default=str).encode("utf-8")
    start_response(f"{status.value} {status.phrase}", [
        ("Content-Type", "application/json; charset=utf-8"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


def respond_no_content(start_response: StartResponse) -> Iterable[bytes]:
    start_response(f"{HTTPStatus.NO_CONTENT.value} {HTTPStatus.NO_CONTENT.phrase}", [])
    return []


def respond_error(
    status: HTTPStatus, message: str, start_response: StartResponse
) -> Iterable[bytes]:
    return respond_json(
        {"error": status.phrase, "status": status.value, "message": message},
        status,
        start_response,
    )


# --- API Application ---

# URL patterns
BOOKS_LIST = re.compile(r"^/api/books$")
BOOKS_DETAIL = re.compile(r"^/api/books/(?P<book_id>\d+)$")


def create_book_api(store: BookStore) -> Callable[..., Iterable[bytes]]:
    """Create a WSGI application for the books REST API."""

    def app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
        path = str(environ.get("PATH_INFO", "/"))
        method = str(environ.get("REQUEST_METHOD", "GET"))

        # --- /api/books ---
        match = BOOKS_LIST.match(path)
        if match:
            if method == "GET":
                books = [asdict(b) for b in store.list_all()]
                return respond_json({"books": books, "count": len(books)},
                                    HTTPStatus.OK, start_response)

            elif method == "POST":
                data = read_json(environ)
                if not data or not isinstance(data, dict):
                    return respond_error(
                        HTTPStatus.BAD_REQUEST, "JSON body required", start_response
                    )
                # Validate required fields
                missing = [f for f in ("title", "author", "year") if f not in data]
                if missing:
                    return respond_error(
                        HTTPStatus.UNPROCESSABLE_ENTITY,
                        f"Missing required fields: {', '.join(missing)}",
                        start_response,
                    )
                book = store.create(
                    title=data["title"],
                    author=data["author"],
                    year=int(data["year"]),
                    isbn=data.get("isbn", ""),
                )
                return respond_json(asdict(book), HTTPStatus.CREATED, start_response)

            else:
                return respond_error(
                    HTTPStatus.METHOD_NOT_ALLOWED,
                    f"Method {method} not allowed on /api/books",
                    start_response,
                )

        # --- /api/books/<id> ---
        match = BOOKS_DETAIL.match(path)
        if match:
            book_id = int(match.group("book_id"))

            if method == "GET":
                book = store.get(book_id)
                if book is None:
                    return respond_error(
                        HTTPStatus.NOT_FOUND,
                        f"Book {book_id} not found",
                        start_response,
                    )
                return respond_json(asdict(book), HTTPStatus.OK, start_response)

            elif method == "PUT":
                data = read_json(environ)
                if not data or not isinstance(data, dict):
                    return respond_error(
                        HTTPStatus.BAD_REQUEST, "JSON body required", start_response
                    )
                book = store.update(book_id, **data)
                if book is None:
                    return respond_error(
                        HTTPStatus.NOT_FOUND,
                        f"Book {book_id} not found",
                        start_response,
                    )
                return respond_json(asdict(book), HTTPStatus.OK, start_response)

            elif method == "PATCH":
                data = read_json(environ)
                if not data or not isinstance(data, dict):
                    return respond_error(
                        HTTPStatus.BAD_REQUEST, "JSON body required", start_response
                    )
                book = store.update(book_id, **data)
                if book is None:
                    return respond_error(
                        HTTPStatus.NOT_FOUND,
                        f"Book {book_id} not found",
                        start_response,
                    )
                return respond_json(asdict(book), HTTPStatus.OK, start_response)

            elif method == "DELETE":
                if not store.delete(book_id):
                    return respond_error(
                        HTTPStatus.NOT_FOUND,
                        f"Book {book_id} not found",
                        start_response,
                    )
                return respond_no_content(start_response)

            else:
                return respond_error(
                    HTTPStatus.METHOD_NOT_ALLOWED,
                    f"Method {method} not allowed",
                    start_response,
                )

        return respond_error(HTTPStatus.NOT_FOUND, "Endpoint not found", start_response)

    return app


print("Book API created. See the next cell for testing.")

## Testing the REST API

We exercise every CRUD operation on our books API: creating, listing,
reading, updating (PUT and PATCH), and deleting resources. Each test
verifies the correct status code and response body.

In [ ]:
import json
import io
from http import HTTPStatus
from typing import Callable, Any, Iterable


# Create a fresh store and API
store = BookStore()
api = create_book_api(store)


def api_request(
    method: str,
    path: str,
    body: dict[str, Any] | None = None,
) -> tuple[str, dict[str, Any] | None]:
    """Make a request to the API and return (status, parsed_body)."""
    environ: dict[str, object] = {
        "REQUEST_METHOD": method,
        "PATH_INFO": path,
    }

    if body is not None:
        encoded = json.dumps(body).encode("utf-8")
        environ["CONTENT_TYPE"] = "application/json"
        environ["CONTENT_LENGTH"] = str(len(encoded))
        environ["wsgi.input"] = io.BytesIO(encoded)

    captured_status = ""

    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal captured_status
        captured_status = s
        return lambda *a: None

    response_body = b"".join(api(environ, sr))
    parsed = json.loads(response_body) if response_body else None
    return captured_status, parsed


# --- Test CRUD Operations ---

print("=== 1. Create Books (POST /api/books) ===")
status, data = api_request("POST", "/api/books", {
    "title": "Fluent Python",
    "author": "Luciano Ramalho",
    "year": 2022,
    "isbn": "978-1492056355",
})
print(f"  {status} -> {json.dumps(data)}")

status, data = api_request("POST", "/api/books", {
    "title": "Python Cookbook",
    "author": "David Beazley",
    "year": 2013,
})
print(f"  {status} -> {json.dumps(data)}")

status, data = api_request("POST", "/api/books", {
    "title": "Effective Python",
    "author": "Brett Slatkin",
    "year": 2019,
})
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 2. List All Books (GET /api/books) ===")
status, data = api_request("GET", "/api/books")
print(f"  {status} -> {data['count']} books")
for book in data["books"]:
    print(f"    [{book['id']}] {book['title']} by {book['author']} ({book['year']})")

print("\n=== 3. Get Single Book (GET /api/books/1) ===")
status, data = api_request("GET", "/api/books/1")
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 4. Get Non-Existent Book (GET /api/books/999) ===")
status, data = api_request("GET", "/api/books/999")
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 5. Update Book (PUT /api/books/2) ===")
status, data = api_request("PUT", "/api/books/2", {
    "title": "Python Cookbook, 3rd Ed.",
    "author": "David Beazley & Brian K. Jones",
    "year": 2013,
})
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 6. Partial Update (PATCH /api/books/3) ===")
status, data = api_request("PATCH", "/api/books/3", {
    "year": 2020,
    "isbn": "978-0134854717",
})
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 7. Delete Book (DELETE /api/books/2) ===")
status, data = api_request("DELETE", "/api/books/2")
print(f"  {status} -> (no body)")

print("\n=== 8. Verify Deletion (GET /api/books) ===")
status, data = api_request("GET", "/api/books")
print(f"  {status} -> {data['count']} books remaining")
for book in data["books"]:
    print(f"    [{book['id']}] {book['title']}")

print("\n=== 9. Validation Error (POST without required fields) ===")
status, data = api_request("POST", "/api/books", {"title": "Incomplete"})
print(f"  {status} -> {json.dumps(data)}")

print("\n=== 10. Delete Non-Existent (DELETE /api/books/999) ===")
status, data = api_request("DELETE", "/api/books/999")
print(f"  {status} -> {json.dumps(data)}")

## Running the API with a Real HTTP Server

Finally, we can serve our REST API over HTTP using `wsgiref` and make real
HTTP requests with `urllib`. This demonstrates the complete end-to-end flow
with middleware stacked on top of the core application.

In [ ]:
from wsgiref.simple_server import make_server
import threading
import urllib.request
import json
from typing import Any


# Create a fresh store and API, wrapped with error handling
live_store = BookStore()
live_api = ErrorHandlingMiddleware(create_book_api(live_store))

# Start the server
httpd = make_server("127.0.0.1", 0, live_api)
port = httpd.server_address[1]
base_url = f"http://127.0.0.1:{port}"
print(f"API server running at {base_url}")

server_thread = threading.Thread(target=httpd.serve_forever, daemon=True)
server_thread.start()


def http_request(
    method: str, path: str, body: dict[str, Any] | None = None
) -> tuple[int, Any]:
    """Make an HTTP request and return (status_code, parsed_json)."""
    url = f"{base_url}{path}"
    data = json.dumps(body).encode("utf-8") if body else None
    req = urllib.request.Request(
        url, data=data, method=method,
        headers={"Content-Type": "application/json"} if data else {},
    )
    try:
        with urllib.request.urlopen(req) as resp:
            body_bytes = resp.read()
            return resp.status, json.loads(body_bytes) if body_bytes else None
    except urllib.error.HTTPError as e:
        body_bytes = e.read()
        return e.code, json.loads(body_bytes) if body_bytes else None


try:
    # Create a book
    print("\n--- POST /api/books ---")
    status, data = http_request("POST", "/api/books", {
        "title": "Architecture Patterns with Python",
        "author": "Harry Percival",
        "year": 2020,
    })
    print(f"  {status}: {json.dumps(data)}")

    # List books
    print("\n--- GET /api/books ---")
    status, data = http_request("GET", "/api/books")
    print(f"  {status}: {data['count']} book(s)")

    # Get the book
    print("\n--- GET /api/books/1 ---")
    status, data = http_request("GET", "/api/books/1")
    print(f"  {status}: {data['title']} by {data['author']}")

    # Update the book
    print("\n--- PATCH /api/books/1 ---")
    status, data = http_request("PATCH", "/api/books/1", {"year": 2021})
    print(f"  {status}: year updated to {data['year']}")

    # Delete the book
    print("\n--- DELETE /api/books/1 ---")
    status, data = http_request("DELETE", "/api/books/1")
    print(f"  {status}: deleted")

    # Verify deletion
    print("\n--- GET /api/books/1 (after delete) ---")
    status, data = http_request("GET", "/api/books/1")
    print(f"  {status}: {data}")

finally:
    httpd.shutdown()
    print("\nServer shut down.")

## Summary

This notebook covered the complete lifecycle of building REST APIs in Python:

1. **REST conventions**: Resources as nouns, HTTP methods as verbs, status codes
   as outcomes (GET=read, POST=create, PUT/PATCH=update, DELETE=remove)
2. **JSON handling**: Parsing request bodies with `json.loads()`, serializing
   responses with `json.dumps()`, and proper Content-Type headers
3. **Content negotiation**: Parsing `Accept` headers and validating `Content-Type`
   to ensure client-server format agreement
4. **Middleware pattern**: Wrapping WSGI apps for cross-cutting concerns --
   logging, authentication, error handling, and CORS
5. **Error handling**: Custom exception hierarchy mapping to HTTP status codes,
   structured JSON error responses, and an error-handling middleware
6. **CORS**: Understanding the Same-Origin Policy, preflight `OPTIONS` requests,
   and the `Access-Control-*` headers
7. **API versioning**: URL path versioning with a version-dispatching router
8. **Complete CRUD API**: An end-to-end books API with routing, validation,
   proper status codes, and real HTTP server integration

These patterns form the foundation that frameworks like Flask, Django REST
Framework, and FastAPI build upon. Understanding them at the WSGI level gives
you deep insight into how web frameworks work internally.